# Phys 260: Python assignment header

### (1) Fill out the cell below.  
The cell below is a **code cell**.  Fill out your University of Michigan uniqname, then your name, and collaborators in the cell below **inside the quotes**.  

**Do not delete the quotes.**  We will use this information to organize your assignments.  To edit and execute cells, double click inside the cell, type, and press \<shift\>+\<enter\> to execute.

In [ ]:
UNIQNAME = ""
NAME = ""
COLLABORATORS = ""

### (2) Check your python version.  
**Execute the cell below** (double click in the cell and press \<shift\>+\<enter\>, or click in the cell and press the Run button) to check that you are using a version of python that is compatible with the tool we are using to grade your assignments.  If your ```IPython``` version is too old, we will *not* be able to grade your assignments.

In [1]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

### (3) Do your best to answer all questions in the assignment.  
To answer questions, **replace** anything that says either
- "YOUR ANSWER HERE" 
- 
```
YOUR CODE HERE
raise NotImplementedError
``` 

with your answer/code.  Cells with either of the two bullet points above are cells of the notebook that will be graded.

**To edit markdown** cells (e.g. this one),  *double click in the cell to type*.  Press \<shift\>+\<enter\> to execute the cell.  Try editing the text below to replace the with your information (e.g. Camille Avestruz, cavestru):  

[first name] [last name], uniqname


### (4) Make sure your notebook runs sequentially.
After you complete this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Introduction -- Reminder

Each Python lab will start with a pre-flight exercise that walks through building some of the set up and tools ($\sim$ 30 min), followed by an in-class tutorial with time for Q+A (50 min) so you can walk through steps that will be necessary for the homework assignment you will submit ($\sim$ 3 hrs).  Each lab will contain starter code, similar to what you see below.  Please fill in the code to complete the pre-flight assignment in preparation for the in-class tutorial.  

Preflight ($\sim$30-60 min, 10 points) **Typically due: Wednesdays 3pm EST**

*Preflight typically graded by Wednesday 5p EST*

In-class tutorial and Q+A ($\sim$ 50 min, 10 points) **Typically occurs: Thursdays 9am EST**

Homework assignment ($\sim$ 3-5 hrs, 30 points) **Typically due: Mondays 9am EST** 

*Homework typically graded by Thursday 5p*

When we grade your homework, we will not run your code. Once submitted, your notebook should have the outputs for all of your results.  Please do not include long outputs from debugging, beyond a few print statements and the requested visualizations (i.e. plots).

**Grading:** When we grade your notebook, we will convert the .ipynb file to an HTML file.  We will be using [nbgrader](https://nbgrader.readthedocs.io/en/stable/) to grade your notebooks.  **Note:** If your ```IPython``` version is too old, we will *not* be able to grade your assignments.


# Phys 260 Python Lab 4: Simulating the Capacitor (30 points total)

## Tutorial/HW summary
- Code review exercise:  relax_2d_potential
- Define boolean statements to identify plate positions
- Multi-axes visualization (i.e. subplots with matplotlib)
- Model the capacitor with a function to allow variations in configuration (i.e. d/L)
- 

## Numerical approaches -- when analytic approaches no longer hold

In the last tutorial/hw, you used the method of relaxation to find the potential inside a rectangular box, where the potential was specified on the sides. While that particular problem can be solved analytically (you will learn how to do this in Physics 405), the method of relaxation allows us to study problems that do not have an analytical solution. 

Consider the familiar example of a parallel-plate capacitor. In most physics classes, we pretend that the electric field between the plates can be approximated by the field of two infinite charged planes, so it's constant in between the plates and zero everywhere outside. Here's a typical illustration of this approximation:

<img src="capacitor.gif" alt="Parallel-plate capacitor" style="width: 200px;"/>

This is an idealization, like frictionless surfaces and masssless springs. The approximation works as long as the linear dimensions of the plates are large compared to the separation: $A\gg d^2$. In this assignment we'll apply this technique to understand what the electric field of a parallel-plate capacitor *really* looks like, and learn how the usual approximation starts to break down as the plate separation increases.

Much of the code in this assignment overlaps with the last tutorial/hw and the recent preflight.

In [2]:
import numpy as np
from matplotlib import pyplot as plt

**Quick group exercise** (2 points -- 2 min):  You'll notice that we define the meshgrid below in terms of the number of points, `num_points_1d`, the indexing, and the `grid_extent` to the left/right, top/bottom.  Define `grid_spacing` in terms of the extent and number of points

In [14]:
#  Generate x,y coordinates on the rectangular grid.
num_points_1d = 100
indexing = 'ij'
grid_extent = 4
x_positions, y_positions = np.meshgrid(np.linspace(-grid_extent,grid_extent,num_points_1d), 
                                       np.linspace(-grid_extent,grid_extent,num_points_1d),
                                       indexing=indexing) 
position_array = np.array([x_positions,y_positions])
potential_array = np.zeros(x_positions.shape)

# Define grid_spacing below
grid_spacing = 2 * grid_extent / num_points_1d

In [15]:
"""Execute to make sure you're on the right track"""
assert(grid_spacing<1)

## Review of relevant functions and code:  Exercise in code review

[*Code review*](https://medium.com/palantir/code-review-best-practices-19e02780015f) is an important component of any software development.  Briefly, code review is the process by which another person (or people) review code for the purposes of helping to debug, make more readable, or otherwise. 

Super aside, the term "bug" in a code comes from the [first incident](https://www.computerhistory.org/tdih/september/9/) when an actual bug was found in a computer (a moth was trapped).  The incident was logged by, and later popularized by, computer-language pioneer Grace Hopper: [Grace Hopper's Bug](https://www.nationalgeographic.org/thisday/sep9/worlds-first-computer-bug/).

Aside from improving code, code review is also a nice opportunity to pick up on tips, tricks, more efficient coding habits, etc.  Like asking and answering questions, it is yet another way to learn from one another.

**Group exercise** (5 min -- 2 points): Below are the definitions of functions `set_boundary_conditions_rectangle` (you saw this previously as `set_boundary_conditions`) and `relax_potential_2d`.  I have inserted bugs in the code for the functions.  Try executing the next three code cells.  You'll find that the `while` loop is stuck.  The bug in `set_boundary_conditions_rectangle` is more of a redundancy (not quite a bug).  

First, identify the redundancy in `set_boundary_conditions_rectangle`, and remove it.  Hint:  The first exercise from the preflight, where we look at how different python objects change (or don't change) when passed to a function, is relevant.  Also, recall how we used `set_boundary_conditions` in the preflight.  


In [16]:
def set_boundary_conditions_rectangle(potential_array, pts_in_meshgrid, 
                                        bounds_left=0, bounds_right=0, 
                                        bounds_top=100, bounds_bottom=-100) :
    """ Returns potential array with set boundary conditions.  Sets in order of x edges, y edges, then z edges (if 3-d)

    Inputs:
    potential_array (n by m-darray) : array containing the potential at each sampled point 
    pts_in_meshgrid (n by m-darray) : x, y coordinates from the grid
    bounds_right (float) : value of potential on right edge
    bounds_left (float) : value of potential on left edge
    bounds_top (float) : value of potential on top edge
    bounds_bottom (float) : value of potential on bottom edge

    Outputs:
    potential_array (n by m-darray) : array containing the potential at each sampled point with boundary conditions set 
    """
    
    assert(potential_array.shape == pts_in_meshgrid[0].shape)
    
    # Separate out the position arrays along each axis for readability
    x_positions = pts_in_meshgrid[0]
    y_positions = pts_in_meshgrid[1]

    # Identify the edges
    right_edge_mask = np.where(x_positions == x_positions.max())
    left_edge_mask = np.where(x_positions == x_positions.min())
    top_edge_mask = np.where(y_positions == y_positions.max())
    bottom_edge_mask = np.where(y_positions == y_positions.min())
    
    # Set the bounds
    potential_array[right_edge_mask] = bounds_right    
    potential_array[left_edge_mask] = bounds_left
    potential_array[top_edge_mask] = bounds_top
    potential_array[bottom_edge_mask] = bounds_bottom
    
    # Returns None (default return of functions)

In [17]:
"""Execute to check you're on the right track"""
test_potential_array = potential_array
set_boundary_conditions_rectangle(test_potential_array, np.array([x_positions, y_positions]))
assert(test_potential_array.shape == potential_array.shape)

**Open question** (2 points -- take home): Why was the original code redundant?

YOUR ANSWER HERE

**Group exercise** (5 min -- 2 points): There are two bugs in the function below.  The first bug is related to the adjustment we made when we removed the redundancy in `set_boundary_conditions`, which assumes `set_boundary_conditions` will be used a certain way.  See what happens before you fix that bug and after you fix that bug.  The second bug will affect how the `while` loop proceeds.  Fix that bug as well.  *Note*: You'll notice some other key differences in this version of `relax_potential_2d`.  Namely, it takes in the boundary conditions function as an argument, and also takes in a generalization of keyword arguments for that function with the `**`.  These are not bugs, but we will talk about these differences.

In [19]:
def relax_potential_2d( potential_array, pts_in_meshgrid, 
                           bc_func=set_boundary_conditions_rectangle,
                            **bc_func_kwargs ) :
    """Returns relaxed 2-d potential array with set boundary conditions.  

    Inputs:
    potential_array (n-darray) : array containing the potential at each sampled point 
    pts_in_meshgrid (n by m-darray) : x, y coordinates from the grid 
    bc_func (function) : function that sets boundary conditions, default set_boundary_conditions_rectangle
    **bc_func_kwarg : any key word arguments for bc_func

    Outputs:
    relaxed_potential (n-darray) : array containing the relaxed potential at each sampled point with boundary conditions set     
    """
    
    # Define potential_left, right, top, and bottom
    potential_bottom = np.roll(potential_array, -1, axis=0)
    potential_top = np.roll(potential_array, 1, axis=0)
    potential_left = np.roll(potential_array, -1, axis=1)
    potential_right = np.roll(potential_array, 1, axis=1)

    relaxed_potential = np.mean(np.array([potential_left, potential_right, 
                                          potential_top, potential_bottom]),axis=0)
    assert(relaxed_potential.shape == potential_array.shape)
    
    
    bc_func(relaxed_potential, pts_in_meshgrid, **bc_func_kwargs)
    
    
    return relaxed_potential

**Open question** (2 points -- take home):  Describe why you needed to change `relax_potential_2d` given the change in `set_boundary_conditions`.  What happened in the loop below?  What was the second change you made?

YOUR ANSWER HERE

In [20]:
convergence_criterion = 0.01
largest_iteration_difference = 1 # initialize

# Initialize the potential
potential_2d = np.zeros(x_positions.shape) # or set_boundary_conditions_2d(np.zeros(x_positions.shape))

# Collect the largest_iteration_difference
largest_iteration_differences = []

while largest_iteration_difference > convergence_criterion :
    relaxed_potential = relax_potential_2d(potential_2d, position_array, 
                                            bc_func=set_boundary_conditions_rectangle,
                                            bounds_left=0, bounds_right=0, 
                                            bounds_top=100, bounds_bottom=-100)

    largest_iteration_difference = np.max(np.abs(relaxed_potential - potential_2d))
    largest_iteration_differences.append(largest_iteration_difference)
    potential_2d = relaxed_potential

print(potential_2d)

[[-100.            0.            0.         ...    0.
     0.          100.        ]
 [-100.          -49.96117278  -30.15713619 ...   30.15713619
    49.96117278  100.        ]
 [-100.          -69.68764127  -49.84486382 ...   49.84486382
    69.68764127  100.        ]
 ...
 [-100.          -69.68764127  -49.84486382 ...   49.84486382
    69.68764127  100.        ]
 [-100.          -49.96117278  -30.15713619 ...   30.15713619
    49.96117278  100.        ]
 [-100.            0.            0.         ...    0.
     0.          100.        ]]


In [ ]:
print(len(largest_iteration_differences))
plt.plot(largest_iteration_differences)
plt.yscale('log')

**Open question** (2 points): 
- Describe what the bug in the code would do in the while loop and why.

YOUR ANSWER HERE

Below, we include the function to calculate the electric field for any potential.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## The case of parallel plate capacitors

The rest of this tutorial/hw will model a parallel-plate capacitor with width $L$ pixels and separation $d$. We will care about how the ratio, $d/L$, impacts the electric field.  So, (1) we will keep $L$ fixed to extend across 1/2 the total width of points sampled and (2) we will vary $d$.

<img src="cap-illustration.png" alt="parallel-plate capacitor" style="width: 400px;"/>

### Define a boolean statement
**Group exercise:** (2 points -- 5 min)  We will assume each plate is sampled by a line of points parallel to the x-axis.  Note, if $L$ is fixed to extend across 1/2 the total width of the points sampled and the plates lie parallel to the x-axis, the plates are sampled by points whose x coordinate values have absolute values less than or equal to half the maximum x coordinate value.  Write the boolean statement that picks out the `plate_x_boolean` of the plates.  `plate_x_boolean` should look like an array of True and False.

In [21]:
# Define plate_x_boolean here
plate_x_boolean = np.abs(x_positions) <= x_positions.max() * 1/2 # replace max with grid_extent


[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [22]:
"""Execute to check you're on the right track"""
assert(plate_x_boolean.shape == x_positions.shape)

**Filling out a function**: (2 points -- take home) The previous exercise identified a 2-d swath containing the plates (where the x-coordinates matched those of the plates).  We will also assume that the plates are equidistant from the x-axis, with a distance as close to $d/2$ as possible (given the meshgrid spacing).  This will give us a line of points corresponding to each plate.  Below, we define a function that returns the `top_y_boolean` and the `bottom_y_boolean`. 

You will need to fill in `find_plate_points`: 
- Define plate_x_boolean using the kwarg `fractional_plate_width`.  In the exercise above, we assumed this is 1/2.  
- Define `plate_y_boolean_top` and `plate_y_boolean_bottom` using the function `find_plate_y_booleans`.

In [ ]:
def find_plate_y_booleans(d, y_position_array) :
    """Returns the booleans corresponding to y positions that are a distance d/2 pixels above and below the x axis
    
    Inputs:
    d (float) : distance separating capacitor plates 
    y_position_array (n by m-darray) : array of the y coordinates of the sampled points

    Outputs:
    top_y_boolean (n by m-darray) : array containing True for y coordinates closest to d/2 
    bottom_y_boolean (n by m-darray) : array containing True for y coordinates closest to -d/2 
    """
    
    assert(d >= grid_spacing)
    
    top_y_boolean = np.abs(y_position_array-d/2) == np.min(np.abs(y_position_array-d/2))
    bottom_y_boolean = np.abs(y_position_array+d/2) == np.min(np.abs(y_position_array+d/2))
    
    return top_y_boolean, bottom_y_boolean

def find_plate_points(d, x_position_array, y_position_array, fractional_plate_width = 1/2) :
    """Returns the indices for top and bottom plates, given d and x/y coordinates.
    
    Inputs:
    d (float) : number of grid points separating capacitor plates 
    x_position_array (n by m-darray) : array of the y coordinates of the sampled points
    y_position_array (n by m-darray) : array of the y coordinates of the sampled points
    fractional_plate_width (float) : fractional width of sampled points that the plate extends across

    Outputs:
    top_plate_indices (tuple) : tuple containing indices of top plate
    bottom_plate_indices (tuple) : tuple containing indices of bottom plate
    """
    
    # Define plate_x_boolean using the kwarg fractional_plate_width below
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Define plate_y_boolean_top and plate_y_boolean_bottom using the function find_plate_y_booleans
    # YOUR CODE HERE
    raise NotImplementedError()

    # Define top_plate_indices and bottom_plate_indices using np.where
    top_plate_indices = np.where(plate_x_boolean & plate_y_boolean_top)
    bottom_plate_indices = np.where(plate_x_boolean & plate_y_boolean_bottom)
        
    return top_plate_indices, bottom_plate_indices

In [ ]:
"""Execute to check you're on the right track"""
assert(type(find_plate_points(0.1, x_positions, y_positions)) == tuple)
assert(type(find_plate_points(0.1, x_positions, y_positions)[0]) == tuple)
assert(len(find_plate_points(0.1, x_positions, y_positions)[0][1]) > \
        len(find_plate_points(0.1, x_positions, y_positions, fractional_plate_width=0.25)[0][1]))

**Set capacitor conditions**: (2 points -- Take home) Fill out the missing line in `set_capacitor_conditions` below.  Make sure to use `find_plate_points` for full credit.

In [ ]:
def set_capacitor_conditions(potential_array, pts_in_meshgrid, 
                                 d=.5, bounds_top=100, bounds_bottom=-100, 
                                 fractional_plate_width=0.5,) :
    """ Modifies potential array with set boundary conditions for a capacitor

    Inputs:
    potential_array (n by m-darray) : array containing the potential at each sampled point 
    pts_in_meshgrid (n by m-darray) : x, y coordinates from the grid 
    d (float) : distance separating capacitor plates 
    bounds_top (float) : value of potential on top plate
    bounds_bottom (float) : value of potential on bottom plate
    fractional_plate_width (float) : fractional width of sampled points that the plate extends across 
    
    Outputs:
    None
    """
    
    x_positions = pts_in_meshgrid[0]
    y_positions = pts_in_meshgrid[1]
    
    assert(potential_array.shape == x_positions.shape)

    # Identify the plate indices, top_plate_mask and bottom_plate_mask
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Set the bounds
    potential_array[bottom_plate_mask] = bounds_bottom
    potential_array[top_plate_mask] = bounds_top


In [ ]:
"""Execute to check you're on the right track"""
test_potential = potential_2d
set_capacitor_conditions(test_potential, position_array, d=.1, fractional_plate_width=0.25)
assert(test_potential.shape == potential_2d.shape)

## Write a function to model the capacitor (5 points -- Take home)

Below, we write a function to model the capacitor.  Note, we've essentially taken the contents of the cell with the `while` loop several cells above, and put that into a function.  The primary reason this is in a function is so we can vary over `d` by simply calling this function again with a different value for the key word argument `d`.  The crucial difference here is in how `relaxed_potential` gets defined in each iteration of the while loop. 

Fill in the necessary code below to define `relaxed_potential` in each iteration.  You'll need to properly call `relax_potential_2d` for the case of the capacitor.  *Hint*: Most of the key word arguments of `model_capacitor` are relevant to `set_capacitor_conditions`.

In [ ]:
def model_capacitor(position_array, convergence_criterion = 0.01, 
                    d=.1, fractional_plate_width=0.5,
                   bounds_top=100, bounds_bottom=-100) :
    """ Models the capacitor with set boundary conditions for a capacitor

    Inputs:
    position_array (2 by n by m-darray) : x, y coordinates from the grid 
    convergence_criterion (float) : fractional width of sampled points that the plate extends across   
    d (float) : distance separating capacitor plates 
    fractional_plate_width (float) : fractional width of sampled points that the plate extends across   
    bounds_top (float) : value of potential on top plate
    bounds_bottom (float) : value of potential on bottom plate
    

    Outputs:
    potential_2d (n by m-darray) : Modeled 2d potential of the capacitor
    """

    largest_iteration_difference = 1 # initialize

    # Initialize the potential
    potential_2d = np.zeros(x_positions.shape) 

    # Collect the largest_iteration_difference
    largest_iteration_differences = []

    while largest_iteration_difference > convergence_criterion :
        # Define the relaxed potential below using relax_potential_2d with the appropriate args and kwargs
        # YOUR CODE HERE
        raise NotImplementedError()

        largest_iteration_difference = np.max(np.abs(relaxed_potential - potential_2d))
        largest_iteration_differences.append(largest_iteration_difference)
        potential_2d = relaxed_potential
    
    print("Modeled with ", len(largest_iteration_differences), "iterations")
    
    return potential_2d



In [ ]:
"""Execute to check you're on the right track"""
test_capacitor = model_capacitor(position_array) 
assert(test_capacitor.shape == x_positions.shape)

## Group Exercise if there is timeMake an nxm figure with a loop (2 points -- 5min)

From the preflight, you made a figure with 1x2 axes.  In the cell below, we show the analog for figure with subplots in a 2x2 configuration where each axis (subplot) shows $f(x) = x^n$, where n ranges from 0 to 3.  For this exercise, make the same 2x2 figure.  However, do this in a `for` loop, and we will iterate over both the iteration number of the axis, `n`, and the axis itself. 

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(14,6))

x = np.arange(20)

n=0
ax1.plot(x, x**n)
ax1.scatter(x, x**n, color='r')
ax1.set_xlabel('x', fontsize='xx-large')

n=1
ax2.plot(x, x**n)
ax2.scatter(x, x**n, color='r')
ax2.set_xlabel('x', fontsize='xx-large')

n=2
ax3.plot(x, x**n)
ax3.scatter(x, x**n, color='r')
ax3.set_xlabel('x', fontsize='xx-large')

n=3
ax4.plot(x, x**n)
ax4.scatter(x, x**n, color='r')
ax4.set_xlabel('x', fontsize='xx-large')


ax1.set_ylabel('f(x)=1', fontsize='xx-large')
ax3.set_ylabel('f(x)=x**2', fontsize='xx-large')    

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(14,6))

x = np.arange(20)

for n, axis in enumerate(axes.flatten()) :
    print(n,axis)
    # YOUR CODE HERE
    raise NotImplementedError()

## Visualizing different configurations of the capacitor

Recall, we had set $L$ to be half of the width of the meshgrid points, meaning it is as wide as the extent.  So, $d/L = d/$`grid_extent`.  We are interested in looking at how the field changes for a capacitor that is $d/L=0.1, 0.2, 0.3, 0.5, 1.0, 2.0$.  Below, define `capacitor_distances` such that `capacitor_distances` is an array of values of d that satisfy the different $d/L$.  This will need to be in terms of `grid_extent`.

In [ ]:
# Define capacitor_distances here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(capacitor_distances.shape == (6,))

## Calculate capacitor potential for each configuration (2 points)

Use a `for` loop to calculate the capacitor potential for each configuration.  Collect in a list called `capacitor_potentials`.


In [ ]:
# Define capacitor_potentials here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(type(capacitor_potentials) == list)

### New trick:  A list comprehension
Below, we use a *list comprehension* to collect the associated electric fields.  Can you see how we might have used this in the previous exercise?

In [ ]:
capacitor_efields = [calc_efield_2d(capacitor_potential) for capacitor_potential in capacitor_potentials]

### Edges: Where the analytic assumptions break down
When working analytically, we assumed that the parallel plate capacitors are close enough to two infinite plates.  That inherently assumes that the electric field between the center of both plates is the same as that on the edges.  Below, we write a boolean statement to identify the center and the edge of the plates along the x-axis.

In [ ]:

center_of_grid_index = np.where((np.abs(x_positions) == np.min(np.abs(x_positions))) & (np.abs(y_positions) == np.min(np.abs(y_positions))))

edge_of_capacitor_index = np.where((np.abs(x_positions-grid_extent/2) == np.min(np.abs(x_positions-grid_extent/2))) & (np.abs(y_positions) == np.min(np.abs(y_positions))))


### Collect the efield values

Below, we completely take advantage of list comprehensions to collect the magnitude of the electric field for each configuration, the magnitudes at the center, and the magnitudes at the edge.       

In [ ]:
efield_magnitudes = [np.linalg.norm(efield, axis=0) for efield in capacitor_efields]
center_efields = [efield_magnitude[center_of_grid_index] for efield_magnitude in efield_magnitudes]
edge_efields = [efield_magnitude[edge_of_capacitor_index] for efield_magnitude in efield_magnitudes]

**Plot deviation from ideal** (2 points):
Plot the deviation from the ideal.  In the cell below, plot the ratio of the E field at the edge to the E field at the center as a function of d/L.  Label your figure, and make sure ratios are correct.

In [ ]:
#  Plot relationship here
# YOUR CODE HERE
raise NotImplementedError()

### Visualize the capacitor 

Below, we visualize the results of our modeled capacitor for the default keyword arguments.

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,8))
levels = [-90, -50, -10, -1, -.1, -.01, .01, .1, 1, 10, 50, 90 ]

capacitor_potential = capacitor_potentials[0]
efield = calc_efield_2d(capacitor_potential)

emagnitude = 10*np.log(np.linalg.norm(efield, axis=0)) # We will color code by this

ax.streamplot(y_positions, x_positions,  
              efield[1], efield[0], 
              color=emagnitude, linewidth=1, cmap=plt.cm.inferno,
              density=2, arrowstyle='->', arrowsize=1.5)

contour1 = ax.contour(y_positions,x_positions,potential_2d, levels=levels,cmap=plt.cm.Reds)
ax.clabel(contour1, fontsize=10, colors='black')  # label the contours

ax.set_aspect('equal')
ax.set_xlabel('y-axis ', fontsize=16)
ax.set_ylabel('x-axis ', fontsize=16)

## Compare with d=2

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,8))
levels = [-90, -50, -10, -1, -.1, -.01, .01, .1, 1, 10, 50, 90 ]

capacitor_potential, iterations = model_capacitor(position_array, d=2) 
efield = calc_efield_2d(capacitor_potential)

emagnitude = 10*np.log(np.linalg.norm(efield, axis=0)) # We will color code by this

ax.streamplot(y_positions, x_positions,  
              efield[1], efield[0], 
              color=emagnitude, linewidth=1, cmap=plt.cm.inferno,
              density=2, arrowstyle='->', arrowsize=1.5)

contour1 = ax.contour(y_positions,x_positions,potential_2d, levels=levels,cmap=plt.cm.Reds)
ax.clabel(contour1, fontsize=10, colors='black')  # label the contours

ax.set_aspect('equal')
ax.set_xlabel('y-axis ', fontsize=16)
ax.set_ylabel('x-axis ', fontsize=16)

**Talk through your plots** (1 point) : Talk through the plots, from the Ratio vs. d/L plot through the two capacitor visualizations above.  Treat the "talk through" as figure captions that you might see in an article.

YOUR ANSWER HERE